## Gotta troubleshoot this model and why its not working
To do so, I'm going to visualize what the training and validation sets look like.

In [ ]:
import torch
from mimo.models.mimo_unet import MimoUnetModel
from mimo.tasks.sen12tp.sen12tp_datamodule import get_datamodule
from argparse import Namespace
import matplotlib.pyplot as plt
import numpy as np

### First load the Laplace one

In [ ]:
args = {
    'seed': 1,
    'checkpoint_path': 'MIMO NDVI Prediction/hhid6a04/checkpoints/last.ckpt',
    'dataset_dir': '/deepskieslab/rnevin/zenodo_data',
    'batch_size': 32,
    'num_loss_function_params': 2,
    'num_subnetworks': 2,
    'filter_base_count': 30,
    'center_dropout_rate': 0.1,
    'final_dropout_rate': 0.1,
    'encoder_dropout_rate': 0.0,
    'core_dropout_rate': 0.0,
    'decoder_dropout_rate': 0.0,
    'loss_buffer_size': 10,
    'loss_buffer_temperature': 0.3,
    'input_repetition_probability': 0.0,
    'batch_repetitions': 1,
    'patch_size': 256,
    'stride': 249,
    'loss': 'laplace_nll',  # Adjust based on your actual loss function
    'weight_decay': 0.0001,
    'learning_rate': 0.0001,
    'num_workers': 30,
    'training_set_percentage': 1.0,
}

args["input"] = ["VV_sigma0", "VH_sigma0"]
args["target"] = ["NDVI"]  # Example target

'''
python scripts/train/train_ndvi.py   --max_epochs 40   --batch_size 32   -t NDVI   -i VV_sigma0   -i VH_sigma0   --project "MIMO_NDVI_Prediction_Gauss"
'''

args = Namespace(**args)  # Convert dictionary to Namespace
dm_laplace = get_datamodule(args)

Look at the train datamodel.

In [ ]:
print(dir(dm_laplace))

In [ ]:
print(dir(dm_laplace.train_dataloader()))

In [ ]:
# Now we can use the model to make predictions
train_data_laplace = dm_laplace.train_dataloader()
val_data_laplace = dm_laplace.val_dataloader()

In [ ]:
print(dir(train_data_laplace.dataset.ds))

In [ ]:
print("Training patches:", train_data_laplace.dataset.ds.patches[:5])
print("Validation patches:", val_data_laplace.dataset.ds.patches[:5])

In [ ]:
def extract_patch_identifiers(patches):
    """Extracts unique identifiers for patches."""
    return set(
        (patch[0]['s1'].parent.name, patch[0]['s1'].stem)  # Extract scene and patch ID
        for patch in patches
    )

# Extract unique identifiers for each set
train_patches = extract_patch_identifiers(train_data_laplace.dataset.ds.patches)
val_patches = extract_patch_identifiers(val_data_laplace.dataset.ds.patches)

# Identify overlap
overlapping_patches = train_patches.intersection(val_patches)

# Print results
print(f"Number of overlapping patches: {len(overlapping_patches)}")
print(f"Number of training patches: {len(train_patches)}")
print(f"Number of val patches: {len(val_patches)}")
if overlapping_patches:
    print("Overlapping patches (sample):", list(overlapping_patches)[:5])